# Loading other data formats with SpikeInterface

Kilosort 4 natively supports data in binary format, `.bin`. The simplest
way to save your data in this format is to load it into memory one chunk at a
time and save it to a `.bin` file using `NumPy's memmap` function. However,
if you aren't comfortable with that process, the `SpikeInterface` package
can load most common electrophysiology formats in a standardized way that makes
it easy to extract the data.

To follow the steps in this notebook, you will first need to install
`SpikeInterface`:
```
    pip install spikeinterface[full]
```

For each data format, `SpikeInterface` has a `read_<format>` utility that loads
the data as a `RecordingExtractor` object, which we can  use to extract the data
and relevant meta information like sampling frequency. The following example
shows the steps for the `Open Ephys` data format. At the bottom of the notebook,
there are notes on how to load several other common formats. For all cells after
the first, all steps should be the same regardless of format.

1. Load the data

In [ ]:
from pathlib import Path
import numpy as np
from spikeinterface.extractors import read_openephys

# Specify the path where the data will be copied to, and where Kilosort 4
# results will be saved.
DATA_DIRECTORY = Path('/home/example_path')  # NOTE: You should change this
DATA_PATH = DATA_DIRECTORY / 'data.bin'
# Create filepath if it doesn't exist
DATA_DIRECTORY.mkdir(parents=True, exist_ok=True)
# Specify path to your existing data
filepath = Path(".../Record_Node_101")       # NOTE: You must change this
# Load existing data with spikeinterface
# NOTE: Open Ephys data can have multiple streams, specify `stream_id` to
#       load different ones.
recording = read_openephys(filepath, stream_id='0')

2. Get information about channel count, sampling frequency, etc.

In [ ]:
# NOTE: Don't use `recording.get_num_channels()` for number of channels, it
#       sometimes includes channels that are not part of the data.
c = recording.get_traces(segment_index=0, start_frame=0, end_frame=1).shape[1]
s = recording.get_num_segments()
fs = recording.get_sampling_frequency()
N = recording.get_total_samples()
dtype = recording.get_dtype()

3. Create a new binary file and copy the data to it 60,000 samples at a time. Depending on your system's memory, you could increase or decrease the number of samples loaded on each iteration.

In [ ]:
y = np.memmap(DATA_PATH, dtype=dtype, mode='w+', shape=(N,c))
NT = 60000  # Number of samples to copy at a time
total_chunks = int(np.ceil(N/NT))

# Copy data to binary file, 60000 samples at a time
# (same as Kilosort's default batch size).
chunk = 0
for k in range(s):
    n = recording.get_num_samples(segment_index=k)
    i = 0 + k*NT
    while i < n:
        if chunk % 50 == 0:
            print(f'Copying chunk {chunk}/{total_chunks} ...')
        j = i + NT if (i + NT) < n else n
        t = recording.get_traces(segment_index=k, start_frame=i, end_frame=j)
        y[i:j,:] = t
        y.flush()
        i += NT
        chunk += 1

4. (Optional) Verify that the data was copied correctly, once again one chunk at a time. If the data is not matched to within 8 decimal places, this cell will raise an `AssertionError`. Note that this process can take almost as long as copying the data.

In [ ]:
new_y = np.memmap(DATA_PATH, dtype=dtype, mode='r', shape=(N,c))
chunk = 0
chunk_limit = np.inf  # np.inf to check all data

for k in range(s):
    n = recording.get_num_samples(segment_index=k)
    i = 0 + k*NT
    while i < n:
        if chunk > chunk_limit:
            break
        if chunk % 50 == 0:
            print(f'Verifying chunk {chunk}/{total_chunks} ...')

        j = i + NT if (i + NT) < n else n
        t = recording.get_traces(segment_index=k, start_frame=i, end_frame=j)
        assert np.all(t == new_y[i:j,:])
        i += NT
        chunk += 1

At this point, it's a good idea to open the Kilosort gui and check that the
data and probe appear to have been loaded correctly and no settings need to be
tweaked. You will need to input the path to the binary datafile, the folder where
results should be saved, and select a probe file.

```python -m kilosort```

From there, you can either launch Kilosort using the GUI or run the
next notebook cell to run it through the API.

5. Run Kilosort (API)

Note that in this case, we don't actually need to specify a probe since it's
the same as the default Neuropixels 1 configuration. For handling different
probe layouts, provide your own .prb file and/or see the tutorial on creating a
new probe file from scratch.

In [ ]:
from kilosort import run_kilosort, default_settings, PROBE_DIR, io
from kilosort.utils import download_probes

# TODO: explain difference between the two channel count settings.
settings = default_settings()
settings['fs'] = fs                      # Sampling rate
settings['NchanTOT'] = c                 # Number of channels
settings['n_chan_bin'] = c               # Number of channels
settings['data_dir'] = DATA_DIRECTORY    # Directory containing binary file.

# NOTE: The only necessary step here is `probe = io.load_probe(probe_path)`.
#       The rest is specific to Kilosort's default probe files.
download_probes()
probe_name = 'neuropixPhase3B1_kilosortChanMap.mat'
probe_path = PROBE_DIR / probe_name
probe = io.load_probe(probe_path)
# NOTE: If the dataset loaded through spikeinterface contains probe information
#       (some .nwb files, for example), you can use `recording.get_probe()`
#       to get the relevant information and then follow the steps at the end of
#       this notebook to save to a .prb file that can be read into Kilosort.

# This command will both run the spike-sorting analysis and save the results to
# `DATA_DIRECTORY`.
ops, st, clu, tF, Wall, similar_templates, is_ref, est_contam_rate = run_kilosort(
    settings=settings, probe=probe, filename=DATA_PATH,
    # dtype=dtype  # TODO: add this option after dtype pull request is approved
    )

# TODO: explain the variables that are returned

Whether you used the gui or the API, the results can now be browsed in Phy from a terminal with:

```phy template-gui <DATA_DIRECTORY>/kilosort4/params.py```

(replacing DATA_DIRECTORY with the appropriate path)

## Instructions for additional data formats

The following cells demonstrate how to load other dataformats using spikeinterface.
Use these code snippets to modify the first cell of this notebook to work with
different datasets.

See [SpikeInterface's documentation](https://spikeinterface.readthedocs.io/en/latest/modules/extractors.html) for additional details.

SpikeGLX

In [ ]:
# NOTE: You do not need to load SpikeGLX data this way. It is already saved in
#       binary format, so you should just point Kilosort 4 to the .bin file.
from spikeinterface.extractors import read_spikeglx
# Provide path to directory containing .bin file.
filepath = Path(".../TEST_20210920_0_g0/")
recording = read_spikeglx(filepath)

Blackrock

In [ ]:
from spikeinterface.extractors import read_blackrock
# Provide path to nsX file, not nev file.
filepath = Path(".../file_spec_3_0.ns6")
recording = read_blackrock(filepath)

Neuralynx

In [ ]:
from spikeinterface.extractors import read_neuralynx
# Provide path to directory containing .Ncs file(s).
filepath = Path("C:/code/ephy_testing_data/neuralynx/BML/original_data/")
recording = read_neuralynx(filepath)

NWB

In [ ]:
from spikeinterface.extractors import read_nwb_recording
# NOTE: SpikeInterface appears to be picky about which NWB formats it will load.
#       If you encounter issues, please reach out for assistance.
# NOTE: You may need to specify additional keyword arguments for
#       `read_nwb_recording`, such as `electrical_series_name`. Any required
#       arguments should be clearly spelled out by an error message.
filepath = Path(".../ecephys_tutorial_v2.5.0.nwb")
recording = read_nwb_recording(filepath)

Intan

In [ ]:
from spikeinterface.extractors import read_intan
# NOTE: You will need to select the appropriate data stream. If you run without
#       specifying `stream_id`, you will get an error message explaining what
#       each stream corresponds to.
filepath = Path(".../intan_rhs_test_1.rhs")
recording = read_intan(filepath, stream_id='0')

##
## Exporting probes from SpikeInterface

To create a new probe file, we can use `ProbeInterface` (a subpackage of `SpikeInterface`).
You will also need `matplotlib` if you want to visualize the probe geometry (recommended).

You can follow the steps in [this ProbeInterface tutorial](https://probeinterface.readthedocs.io/en/main/examples/ex_01_generate_probe_from_sratch.html)
to create a new probe from scratch, or to plot a probe to check that it is
configured correctly.

Then use the following steps to export to a .prb file that can be read by
Kilosort4.

In [ ]:
from probeinterface import ProbeGroup, write_prb

probe = ...  # From SpikeInterface tutorial, or recording.get_probe()

# Multiple probes can be added to a ProbeGroup. We only have one, but a
# ProbeGroup wrapper is still necessary for `write_prb` to work.
pg = ProbeGroup()
pg.add_probe(probe)
# CHANGE THIS PATH to wherever you want to save your probe file.
write_prb('.../test_prb.prb', pg)

Note that the probe object must have channel indices specified in order to save
to a .prb file. If `write_prb` results in an error indicating these are not set,
you can use the `probe.set_device_channel_indices` method to set them. For example,
for a 24-channel probe with all contacts connected:

In [ ]:
# Must set channel indices for .prb files.
# Indicate "not connected" with a value of -1.
probe.set_device_channel_indices(np.arange(24))